In [ ]:
import pennylane.numpy as np
import pennylane as qml
import tensorflow as tf
from tensorflow.keras import activations
from tensorflow.keras.callbacks import ReduceLROnPlateau
from silence_tensorflow import silence_tensorflow
silence_tensorflow()

In [ ]:
fold = 0
TrainX = np.load(f"TrainX_fold_{fold}.npy", allow_pickle=True)
TrainY = np.load(f"TrainY_fold_{fold}.npy", allow_pickle=True)

ValidationX = np.load(f"ValidationX_fold_{fold}.npy", allow_pickle=True)
ValidationY = np.load(f"ValidationY_fold_{fold}.npy", allow_pickle=True)


In [ ]:
def Data_AngleEmbedding_layer(inputs,n_qubits):
    qml.templates.AngleEmbedding(inputs,rotation='Y', wires=range(n_qubits)) 
    
def ROT_layer(w,qubit_number):
    for i in range(qubit_number):
        qml.Rot(*w[i],wires=i)

def U3_layer(w,qubit_number):
    for i in range(qubit_number):
        qml.U3(*w[i],wires=i)

def strong_entangling_layer(nqubits):
    for i in range(nqubits-1):
        qml.CNOT(wires=[i,i+1])
    qml.CNOT(wires=[nqubits-1,0])

In [ ]:
n_qubit = 3
n_layers=2

weight_shapes = {"weights_1": (n_layers,n_qubit,3),"weights_2": (n_layers,n_qubit,3)}
dev = qml.device('default.qubit', wires=n_qubit)
@qml.qnode(dev)
def qnode(inputs,weights_1,weights_2):
    Data_AngleEmbedding_layer(np.arctan(inputs),n_qubit)
    for k in range(n_layers):
        U3_layer(weights_1[k],n_qubit)
        strong_entangling_layer(n_qubit)
    ROT_layer(weights_2[k],n_qubit) 
    return [qml.expval(qml.PauliZ(wires=i)) for i in range(n_qubit)]

In [ ]:
qlayer = qml.qnn.KerasLayer(qnode, weight_shapes, output_dim=n_qubit)
Activation=tf.keras.layers.Activation(activations.relu)

inputs=tf.keras.Input(shape=(3,))
x1=qlayer(inputs)
x2=qlayer(inputs**2)
x3=tf.keras.layers.Concatenate()([x1, x2])
x4=tf.keras.layers.Activation(activations.relu)(x3)
outputs=tf.keras.layers.Dense(1)(x4)
model=tf.keras.Model(inputs=inputs,outputs=outputs)

opt = tf.keras.optimizers.Adam(learning_rate=0.1)
model.compile(opt, loss="mae")

In [ ]:
re=ReduceLROnPlateau(monitor='loss', factor=0.1, patience=5, verbose=1, mode='min', min_lr=0.00001)
fitting = model.fit(TrainX, TrainY, epochs=100, batch_size=1, validation_data=(ValidationX,ValidationY),callbacks=[re], verbose=2)  

In [ ]:
TestX=np.load("TestX.npy",allow_pickle=True)
TestY=np.load("TestY.npy",allow_pickle=True)

In [ ]:
y_predict=model.predict(TestX,verbose=1)
y_predict=y_predict.T
y_predict=y_predict[0]

In [ ]:
from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(TestY, y_predict))
print('Mean Squared Error:', metrics.mean_squared_error(TestY, y_predict))
print('R^2:', metrics.r2_score(TestY, y_predict))
print("Standard Deviation:",np.std(y_predict))